In [1]:
import os
import pickle
from wanikani import *

In [3]:
# DIC_RAD={}
# DIC_KAN={}
# DIC_VOC={}
# DIC_ALL={}
# DIC_NAMES={}
with open('FIT_DATAS.pkl', 'rb') as inp:
    DIC_RAD,DIC_KAN,DIC_VOC,DIC_ALL,DIC_NAMES,accLVL,accReview_from=pickle.load(inp)

In [4]:
STAGE=0
LVL=4

DATA PROCESSING:

In [5]:
def handle_rads(line):
    line=line.replace('\n','')
    meanings,mnemonics=line.split('.',1)
    meanings = (lambda x: [i.lower() for i in x])(meanings.split(','))
    for i in range(len(meanings)):
        if meanings[i][0]==' ':
            meanings[i]=meanings[i][1:]
        if meanings[i][-1]==' ':
            meanings[i]=meanings[i][:-1]
    if mnemonics[0]==' ':
        mnemonics=mnemonics[1:]
    return meanings,mnemonics
def handle_not_rads(line):
    line=line.replace('\n','')
    meanings,readings,mnemonics = line.split(".", 2)
    meanings = (lambda x: [i.lower() for i in x])(meanings.split(','))
    readings = readings.split(',')
    for i in range(len(meanings)):
        if meanings[i][0]==' ':
            meanings[i]=meanings[i][1:]
        if meanings[i][-1]==' ':
            meanings[i]=meanings[i][:-1]
    if mnemonics[0]==' ':
        mnemonics=mnemonics[1:]
    for i in range(len(readings)):
        if readings[i][0]==' ':
            readings[i]=readings[i][1:]
        if readings[i][-1]==' ':
            readings[i]=readings[i][:-1]
    return meanings,readings,mnemonics

#--------------------------------------------------------------------------------------------------------------------------------------------------------------------------

with open('rads/lvl'+str(LVL)+'/data.txt', 'r') as file:
    data = file.readlines()
RADS_DATA={}
for i in data:
    # data[i]=data[i].replace('\n','')
    # meanings,mnemonics = data[i].split(".", 1)
    # meanings = (lambda x: [i.lower() for i in x])(meanings.split(','))
    meanings,mnemonics=handle_rads(i)
    RADS_DATA[meanings[0]] = [meanings, mnemonics]
    
with open('kans/lvl'+str(LVL)+'/data.txt', 'r') as file:
    data = file.readlines()
KANS_DATA={}
for i in data:
    # data[i]=data[i].replace('\n','')
    # meanings,readings,mnemonics = data[i].split(".", 2)
    # meanings = (lambda x: [i.lower() for i in x])(meanings.split(','))
    meanings,readings,mnemonics = handle_not_rads(i)
    KANS_DATA[meanings[0]] = [meanings,readings, mnemonics]

with open('vocs/lvl'+str(LVL)+'/data.txt', 'r') as file:
    data = file.readlines()
VOCS_DATA={}
for i in range(len(data)):
    data[i]=data[i].replace('\n','')
    meanings,readings,mnemonics = data[i].split(".", 2)
    meanings = (lambda x: [i.lower() for i in x])(meanings.split(','))
    VOCS_DATA[meanings[0]] = [meanings,readings.split(','), mnemonics] 

MATCHING DATA+PICTURES AND INSERTING INTO THE DICTIONARIES

In [6]:
for dirname, _, filenames in os.walk('rads/lvl'+str(LVL)):
    for filename in filenames:
            filepath = os.path.join(dirname, filename)
            if (filename!='.DS_Store')and(filename!='data.txt')and(('rad',filename[:-4].lower()) not in DIC_NAMES):
                #stage, pic, meaning, mnemonics='obvious', lvl=0, previous_time=time.time()
                item=rad(LVL,filepath.lower(),RADS_DATA[filename[:-4].lower()][0],RADS_DATA[filename[:-4].lower()][1],STAGE)
                DIC_RAD[LVL]=DIC_RAD.get(LVL,[])+[item]
                DIC_ALL[LVL]=DIC_ALL.get(LVL,[])+[item]
                DIC_NAMES[('rad',filename[:-4].lower())]=item

for dirname, _, filenames in os.walk('kans/lvl'+str(LVL)):
    for filename in filenames:
            filepath = os.path.join(dirname, filename)
            if (filename!='.DS_Store')and(filename!='data.txt')and(('kan',filename[:-4].lower()) not in DIC_NAMES):
                data=KANS_DATA[filename[:-4].lower()]
                #stage, pic, meanings, readings, mnemonics='obvious', lvl=0, previous_time=time.time()
                item=kan(LVL,filepath.lower(),data[0],data[1],data[2],STAGE)
                DIC_KAN[LVL]=DIC_KAN.get(LVL,[])+[item]
                DIC_ALL[LVL]=DIC_ALL.get(LVL,[])+[item]
                DIC_NAMES[('kan',filename[:-4].lower())]=item

for dirname, _, filenames in os.walk('vocs/lvl'+str(LVL)):
    for filename in filenames:
            filepath = os.path.join(dirname, filename)
            if (filename!='.DS_Store')and(filename!='data.txt')and(('voc',filename[:-4].lower()) not in DIC_NAMES):
                data=VOCS_DATA[filename[:-4].lower()]
                #stage, pic, meanings, readings, mnemonics='obvious',lvl=0,previous_time=time.time()
                item=voc(LVL,filepath.lower(),data[0],data[1],data[2],STAGE)
                DIC_VOC[LVL]=DIC_VOC.get(LVL,[])+[item]
                DIC_ALL[LVL]=DIC_ALL.get(LVL,[])+[item]
                DIC_NAMES[('voc',filename[:-4].lower())]=item

CHECK

In [7]:
DIC_NAMES[('voc','baby')].show()
DIC_NAMES[('voc','another person')].info()
time.sleep(5)
clear_output()

SAVE CHANGES

In [8]:
FIT_DATAS=[DIC_RAD,DIC_KAN,DIC_VOC,DIC_ALL,DIC_NAMES,accLVL,accReview_from]
with open('FIT_DATAS.pkl', 'wb') as outp:
    pickle.dump(FIT_DATAS, outp, pickle.HIGHEST_PROTOCOL)